In [12]:
import numpy as np
np.set_printoptions(suppress=True,precision=7)

# Лабораторна робота №2

## Цільова функція:

$$11x^2 + 14y^2 + z^2 + 0,01xy - 0,001yz - y$$

In [20]:
target_function = lambda x: 11*x[0]**2 + 14*x[1]**2 + x[2]**2 + 0.01*x[0]*x[1] - 0.001*x[1]*x[2] - x[1]

## Умови зупинки:

$$||x^{k+1} - x^k||\leq\epsilon$$

In [21]:
def x_norm_stop(x_prev,x_cur,epselon):
    return np.linalg.norm(x_prev-x_cur) < epselon

$$||f(x^{k+1}) - f(x^k)||\leq\epsilon$$

In [22]:
def func_abs_stop(func_prev,func_cur,epselon):
    return abs(func_prev - func_cur) < epselon

$$||f'(x^{k+1})||\leq\epsilon$$

In [23]:
def grad_abs_stop(grad,epselon):
    return np.linalg.norm(grad)<epselon

## Метод Ньютона

In [24]:
class NeutonMethod:
    
    def __init__(self, target_func, initial_x, grad_step_size, 
                 step_size=1,  adaptive_beta=0, adaptive_epselon=0, fastest_descent_eps=0):
        self.f = target_func
        self.x = initial_x
        
        self.f_value = self.f(self.x)
        
        self.grad_step_size = grad_step_size
        self.step_size = step_size
        self.adaptive_beta = adaptive_beta
        self.adaptive_epselon = adaptive_epselon
        self.initial_step_size = step_size
        self.fastest_descent_eps = fastest_descent_eps
        
        self.grad = None
        self.gesse_matrix = None
        
    @staticmethod
    def first_partial_deriv(f, x, h, var_num):
        x_back, x_forward = x.copy(), x.copy()
        
        # Increase x_back in such a way: (x-h;y)
        x_back[var_num] = x_back[var_num] - h 
        # Increase x_forward in such a way: (x+h;y)
        x_forward[var_num] = x_forward[var_num] + h
        
        return (f(x_forward) - f(x_back))/(2*h)
    
    @staticmethod
    def compose_grad_vec(f, x, h):
        # Compose vector from partial derivatives
        return np.array([NeutonMethod.first_partial_deriv(f,x,h,i) for i in range(x.shape[0])])
    
    @staticmethod
    def second_partial_deriv(f, x, h, var_1_num, var_2_num):
        x_back_back, x_back_for, x_for_back, x_for_for = x.copy(), x.copy(), x.copy(), x.copy()
        
        # Increase x_back_back in such a way: (x-h;y-h)
        x_back_back[var_1_num] = x_back_back[var_1_num] -h
        x_back_back[var_2_num] = x_back_back[var_2_num] -h
        
        # Increase x_back_for in such a way: (x-h;y+h)
        x_back_for[var_1_num] = x_back_for[var_1_num] -h
        x_back_for[var_2_num] = x_back_for[var_2_num] +h
        
        # Increase x_for_back in such a way: (x+h;y-h)
        x_for_back[var_1_num] = x_for_back[var_1_num] +h
        x_for_back[var_2_num] = x_for_back[var_2_num] -h
        
        # Increase x_for_for in such a way: (x+h;y+h)
        x_for_for[var_1_num] = x_for_for[var_1_num] +h
        x_for_for[var_2_num] = x_for_for[var_2_num] +h
        
        return (f(x_back_back) - f(x_back_for) - f(x_for_back) + f(x_for_for))/(4*h**2)
    
    @staticmethod
    def compose_gesse_matrix(f, x, h):
        gesse_matrix = np.zeros((x.shape[0],x.shape[0]))
        
        # Fill elements only under and in diagonal and copy elements upper diagonal (Gesse matrix is semetric)
        for i in range(gesse_matrix.shape[0]):
            for j in range(i+1):
                gesse_matrix[i,j] = NeutonMethod.second_partial_deriv(f,x,h,i,j)
                gesse_matrix[j,i] = gesse_matrix[i,j]
                
        return gesse_matrix
    
    '''
    @staticmethod
    def adaptive_step_size(f, current_f_value, alpha, beta):
        # decrease alpha_k until f(x_k) > f(x_k - alpha_k * inversed(f(x_k)'')f(x_k)')
        while current_f_value < f(alpha):
            alpha = alpha * beta
            
        return alpha
    '''
    
    @staticmethod
    def adaptive_step_size(f, current_f_value, grad, direction, alpha, beta, epselon):
        while f(alpha) - current_f_value > epselon*alpha*(grad@direction):
            alpha = alpha * beta
            
        return alpha

    
    @staticmethod
    def gold_section_search(f, a, b, eps):
        phi = 0.5 * (1.0 + 5.0**0.5)
        
        while abs(b-a) >= eps:
            x_1 = b - (b-a)/phi
            x_2 = a + (b-a)/phi
            
            if f(x_1) > f(x_2):
                a = x_1
            else:
                b = x_2
                
        return (a+b)/2
        
    def backward(self):
        self.grad = NeutonMethod.compose_grad_vec(self.f, self.x, self.grad_step_size) 
        self.gesse_matrix = NeutonMethod.compose_gesse_matrix(self.f, self.x, self.grad_step_size)
        
    def zero_grad(self):
        self.grad = np.zeros(self.x.shape[0])
        self.gesse_matrix = np.zeros((self.x.shape[0],self.x.shape[0]))
        
    def step(self):
        '''
        self.step_size = NeutonMethod.second_adaptive(f=lambda alpha: self.f(self.x - alpha * np.linalg.inv(self.gesse_matrix) @ self.grad),
                                                         current_f_value=self.f_value, grad=self.grad,
                                                         direction=-np.linalg.inv(self.gesse_matrix)@self.grad,
                                                         alpha=self.initial_step_size, beta=self.adaptive_beta,
                                                         epselon=0.25)
        
        '''
        # if beta > 0 and epselon > 0 we are using adaptive step_size
        if self.adaptive_beta > 0 and self.adaptive_epselon > 0:
            self.step_size = NeutonMethod.adaptive_step_size(f=lambda alpha: self.f(self.x - alpha * np.linalg.inv(self.gesse_matrix) @ self.grad),
                                                             current_f_value=self.f_value, grad=self.grad,
                                                             direction=-np.linalg.inv(self.gesse_matrix)@self.grad,
                                                             alpha=self.initial_step_size, beta=self.adaptive_beta,
                                                             epselon=self.adaptive_epselon)
        
        # if fastest descent epselon > 0 we are using fastest descent algorithm
        elif self.fastest_descent_eps > 0:
            self.step_size = NeutonMethod.gold_section_search(f=lambda alpha: self.f(self.x - alpha * np.linalg.inv(self.gesse_matrix) @ self.grad),
                                                              a=0, b=self.initial_step_size, 
                                                              eps=self.fastest_descent_eps)
        
        # x_k+1 = x_k - alpha_k * inversed(f(x_k)'')f(x_k)'
        self.x = self.x - self.step_size * np.linalg.inv(self.gesse_matrix) @ self.grad
        self.f_value = self.f(self.x)
        
    def info(self):
        print('Current x: {}'.format(self.x))
        print('Current f(x): {}'.format(self.f_value))
        print('Current grad: {}'.format(self.grad))
        print('Current gesse matrix:\n {}'.format(self.gesse_matrix))
        print('Step size: {}'.format(self.step_size))
        print('Gradient step size: {}'.format(self.grad_step_size))

Застосуємо метод з величиною кроку = 1

In [27]:
neuton_descent = NeutonMethod(target_func=target_function,
                              initial_x=np.array([1000.,1000.,1000.]),
                              grad_step_size=1)

In [28]:
eps = 0.001
num_itter = 0
previous_x = neuton_descent.x + eps + 1

while not grad_abs_stop(NeutonMethod.compose_grad_vec(f=neuton_descent.f, 
                                                      h=neuton_descent.step_size, 
                                                      x=neuton_descent.x), eps):
    num_itter +=1
    print('\nItteration: {}'.format(num_itter))
    
    previous_x = neuton_descent.x
    neuton_descent.zero_grad()
    # Compute gradient and gesse matrix
    neuton_descent.backward()
    # x_k+1 = x_k + h_k
    neuton_descent.step()
    neuton_descent.info()
    
print('\nConverged in {} itterations'.format(num_itter))


Itteration: 1
Current x: [-0.0000162  0.0357144  0.0000187]
Current f(x): -0.01785714607392642
Current grad: [22010. 28008.  1999.]
Current gesse matrix:
 [[22.     0.01   0.   ]
 [ 0.01  28.    -0.001]
 [ 0.    -0.001  2.   ]]
Step size: 1
Gradient step size: 1

Converged in 1 itterations


In [27]:
egin_values = np.linalg.eig(NeutonMethod.compose_gesse_matrix(f=neuton_descent.f, h=neuton_descent.step_size, x=neuton_descent.x))[0]
egin_values

array([15679999.0000513,      967.9920921,        7.9992852])

In [28]:
abs(egin_values.min()/egin_values.max())

5.101585282973253e-07

Застосуємо метод з постійною величиною кроку

In [ ]:
neuton_descent = NeutonMethod(target_func=target_function,
                              initial_x=np.array([10.,10.,10.]),
                              grad_step_size=0.00001,
                              step_size=0.1)

In [ ]:
eps = 0.00001
num_itter = 0
previous_x = neuton_descent.x + eps + 1

while not grad_abs_stop(NeutonMethod.compose_grad_vec(f=neuton_descent.f, 
                                                      h=neuton_descent.step_size, 
                                                      x=neuton_descent.x), eps):
    num_itter +=1
    print('\nItteration: {}'.format(num_itter))
    
    previous_x = neuton_descent.x
    neuton_descent.zero_grad()
    # Compute gradient and gesse matrix
    neuton_descent.backward()
    # x_k+1 = x_k + h_k
    neuton_descent.step()
    neuton_descent.info()
    
print('\nConverged in {} itterations'.format(num_itter))

Застосуємо метод з адаптивною величиною кроку

In [ ]:
neuton_descent = NeutonMethod(target_func=target_function,
                              initial_x=np.array([10.,10.,10.]),
                              grad_step_size=0.001,
                              step_size=1,
                              adaptive_beta=0.5,
                              adaptive_epselon=0.25)

In [ ]:
eps = 0.001
num_itter = 0
previous_x = neuton_descent.x + eps + 1

while not grad_abs_stop(NeutonMethod.compose_grad_vec(f=neuton_descent.f, 
                                                      h=neuton_descent.step_size, 
                                                      x=neuton_descent.x), eps):
    num_itter +=1
    print('\nItteration: {}'.format(num_itter))
    
    previous_x = neuton_descent.x
    neuton_descent.zero_grad()
    # Compute gradient and gesse matrix
    neuton_descent.backward()
    # x_k+1 = x_k + h_k
    neuton_descent.step()
    neuton_descent.info()
    
print('\nConverged in {} itterations'.format(num_itter))

Застосуємо метод найшвидшого спупску

In [ ]:
neuton_descent = NeutonMethod(target_func=target_function,
                              initial_x=np.array([1.,1.,1.]),
                              step_size=1,
                              grad_step_size=0.00001,
                              fastest_descent_eps=0.00001)

In [ ]:
eps = 0.001
num_itter = 0
previous_x = neuton_descent.x + eps + 1

while not grad_abs_stop(NeutonMethod.compose_grad_vec(f=neuton_descent.f,
                                                      h=neuton_descent.step_size,
                                                      x=neuton_descent.x), eps):
    num_itter +=1
    print('\nItteration: {}'.format(num_itter))
    
    previous_x = neuton_descent.x
    neuton_descent.zero_grad()
    # Compute gradient and gesse matrix
    neuton_descent.backward()
    # x_k+1 = x_k + h_k
    neuton_descent.step()
    neuton_descent.info()
    
print('\nConverged in {} itterations'.format(num_itter))